In [1]:
from datasets import load_dataset

dataset = load_dataset("rungalileo/20_Newsgroups_Fixed")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/rajabyfa/.cache/huggingface/datasets/rungalileo___csv/rungalileo--20_Newsgroups_Fixed-edf414ecc72dd622/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def trim_dataset(dataset, key):
    labels = {}
    for item in dataset[key]:
        if item['label'] not in labels:
            labels[item['label']] = 0
        labels[item['label']] += 1
    print(labels)
    before_len = len(dataset[key])
    print(before_len)
    dataset[key] = dataset[key].filter(lambda example: example['label'] and example['label'] != "None" and example['text'] and len(example["text"]) >= 10)
    after_len = len(dataset[key])
    print(after_len)
    print("the number of removed items : ", before_len - after_len)
    labels = {}
    for item in dataset[key]:
        if item['label'] not in labels:
            labels[item['label']] = 0
        labels[item['label']] += 1
    print(labels)
    return dataset

In [3]:
dataset = trim_dataset(dataset, "train")
dataset = trim_dataset(dataset, "test")

{'rec.autos': 539, 'comp.sys.mac.hardware': 537, 'comp.graphics': 551, 'sci.space': 564, 'talk.politics.guns': 526, 'sci.med': 572, 'comp.sys.ibm.pc.hardware': 562, 'comp.os.ms-windows.misc': 555, 'rec.motorcycles': 551, 'talk.religion.misc': 339, 'None': 650, 'misc.forsale': 565, 'alt.atheism': 449, 'sci.electronics': 563, 'comp.windows.x': 576, 'rec.sport.hockey': 559, 'rec.sport.baseball': 547, 'soc.religion.christian': 582, 'talk.politics.mideast': 521, 'talk.politics.misc': 438, 'sci.crypt': 568}
11314


Filter:   0%|          | 0/11314 [00:00<?, ? examples/s]

10664
the number of removed items :  650
{'rec.autos': 539, 'comp.sys.mac.hardware': 537, 'comp.graphics': 551, 'sci.space': 564, 'talk.politics.guns': 526, 'sci.med': 572, 'comp.sys.ibm.pc.hardware': 562, 'comp.os.ms-windows.misc': 555, 'rec.motorcycles': 551, 'talk.religion.misc': 339, 'misc.forsale': 565, 'alt.atheism': 449, 'sci.electronics': 563, 'comp.windows.x': 576, 'rec.sport.hockey': 559, 'rec.sport.baseball': 547, 'soc.religion.christian': 582, 'talk.politics.mideast': 521, 'talk.politics.misc': 438, 'sci.crypt': 568}
{'rec.autos': 354, 'comp.windows.x': 370, 'None': 513, 'talk.politics.mideast': 353, 'talk.religion.misc': 230, 'sci.med': 371, 'soc.religion.christian': 380, 'comp.graphics': 370, 'comp.os.ms-windows.misc': 369, 'rec.motorcycles': 361, 'alt.atheism': 290, 'comp.sys.mac.hardware': 361, 'misc.forsale': 368, 'talk.politics.guns': 337, 'sci.space': 368, 'comp.sys.ibm.pc.hardware': 374, 'sci.crypt': 360, 'rec.sport.baseball': 366, 'rec.sport.hockey': 378, 'talk.pol

Filter:   0%|          | 0/7532 [00:00<?, ? examples/s]

7019
the number of removed items :  513
{'rec.autos': 354, 'comp.windows.x': 370, 'talk.politics.mideast': 353, 'talk.religion.misc': 230, 'sci.med': 371, 'soc.religion.christian': 380, 'comp.graphics': 370, 'comp.os.ms-windows.misc': 369, 'rec.motorcycles': 361, 'alt.atheism': 290, 'comp.sys.mac.hardware': 361, 'misc.forsale': 368, 'talk.politics.guns': 337, 'sci.space': 368, 'comp.sys.ibm.pc.hardware': 374, 'sci.crypt': 360, 'rec.sport.baseball': 366, 'rec.sport.hockey': 378, 'talk.politics.misc': 292, 'sci.electronics': 367}


In [4]:
from transformers import RobertaTokenizer, BertTokenizer

# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
# comp.graphics
# comp.os.ms-windows.misc
# comp.sys.ibm.pc.hardware
# comp.sys.mac.hardware
# comp.windows.x
# rec.autos
# rec.motorcycles
# rec.sport.baseball
# rec.sport.hockey
# sci.crypt
# <sci.electronics
# sci.med
# sci.space
# misc.forsale
# talk.politics.misc
# talk.politics.guns
# talk.politics.mideast
# talk.religion.misc
# alt.atheism
# soc.religion.christian
# None

import torch


def collate_label_set(data):
    level1 = [
        "comp.os", "comp.sys", "comp.windows", "comp.graphics", "rec.motorcycles", "rec.sport", "rec.autos", "talk.religion",
        "sci.electronics", "sci.med", "sci.space", "misc.forsale", "talk.politics", "sci.crypt", "alt.atheism", "soc.religion"
    ]
    level2_pass = {
        "comp.windows", "comp.os", "talk.religion", "soc.religion"
    }
    level2 = [
        "misc", "guns", "ibm", "mac", "baseball", "hockey", "mideast", "None"
    ]
    for item in data:
        if item['text'] is None:
            item['text'] = " "
        try:
            label = item['label']
            if not label:
                label = "None"
                item['level1'] = level1.index(label)
                item['level2'] = level2.index("None")
            else:
                label = label.split(".")
                label_two = ".".join(label[:2])
                if label_two not in level1:
                    print(label[0])
                else:
                    item['level1'] = level1.index(label_two)
                if len(label) > 2:
                    if label_two in level2_pass:
                        item['level2'] = level2.index("None")
                    else:
                        if label[2] not in level2:
                            print(label[2])
                            item['level2'] = level2.index("None")
                        else:
                            item['level2'] = level2.index(label[2])
                else:
                    item['level2'] = level2.index("None")
        except:
            print(label, item['text'])
            raise
            
    final_data = {"id": None, "text": None, "level1": None, "level2": None}
    final_data['id'] = torch.tensor([item['id'] for item in data])
    final_data['text'] = [item['text'] for item in data]
    final_data['level1'] = torch.tensor([item['level1'] for item in data])
    final_data['level2'] = torch.tensor([item['level2'] for item in data])
    try:
        x = tokenizer.batch_encode_plus(final_data['text'], return_tensors="pt", padding="longest", max_length=512, truncation=True)
    except:
        print(final_data['text'])
        raise
    for key in x:
        final_data[key] = x[key]
        
    return final_data

In [6]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset['train'], batch_size=36, collate_fn=collate_label_set)
test_loader = DataLoader(dataset['test'], batch_size=36, collate_fn=collate_label_set)

In [7]:
from transformers import AutoTokenizer, RobertaModel, BertModel

model_roberta = BertModel.from_pretrained("bert-base-uncased")
# item = next(iter(loader))
# outputs = model_roberta(item['input_ids'], item['attention_mask'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import torch.nn as nn
import torch.nn.functional as F

class NewsRepModule(nn.Module):
    def __init__(self, roberta):
        super().__init__()
        self.roberta = roberta
        self.drop_layer = nn.Dropout(p=0.6)

    def forward(self, input_ids, attention_mask):
        output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.pooler_output
        logits = self.drop_layer(logits)
        return logits

class Level2Calssification(nn.Module):
    def __init__(self,):
        super().__init__()
        self.classification = nn.Linear(768, 8)

    def forward(self, logits):
        _out = self.classification(logits)
        return _out
    
class Level1Calssification(nn.Module):
    def __init__(self,):
        super().__init__()
        self.classification = nn.Linear(768, 50)
        self.relu = nn.LeakyReLU()
        self.classification2 = nn.Linear(50, 16)
        

    def forward(self, logits):
        _out = self.classification(logits)
        _out = self.relu(_out)
        _out = self.classification2(_out)
        return _out


device = "cuda:2"
rep_model = NewsRepModule(model_roberta)
rep_model = rep_model.to(device)

level1_classification = Level1Calssification().to(device)
level2_classification = Level2Calssification().to(device)

In [10]:
params = list(rep_model.parameters()) + list(level1_classification.parameters())
params = list(params) + list(level2_classification.parameters())
optim = torch.optim.AdamW(params, lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [11]:
for item in loader:
    pass

In [10]:
from tqdm import tqdm
for epoch in range(4):
    rep_model.train()
    level1_classification.train()
    level2_classification.train()
    total_loss = 0
    for train_batch in tqdm(loader):
        try:
            rep = rep_model(train_batch['input_ids'].to(device),
                                           train_batch['attention_mask'].to(device))
            level1 = level1_classification(rep)
            level2 = level2_classification(rep)
            
        except:
            print(train_batch['text'])
            raise
        try:
            loss = 0 
            loss += criterion(level1, train_batch['level1'].long().to(device))
            loss += criterion(level2, train_batch['level2'].long().to(device))
            total_loss += loss.item()
            loss.backward()
        except:
            print(level1, train_batch['level1'])
            print(level2, train_batch['level2'])
            raise
        optim.step()
        optim.zero_grad()
    print(f"The loss of epoch {epoch} is {total_loss} \n")
    
#     rep_model.eval()
#     level1_classification.eval()
#     level2_classification.eval()
#     level3_classification.eval()
#     level1_correct, level2_correct, level3_correct = 0, 0, 0
#     level1_correct_actual, level2_correct_actual, level3_correct_actual = 0, 0, 0
#     level1_total, level2_total, level3_total = 0, 0, 0
#     level1_total_actual, level2_total_actual, level3_total_actual = 0, 0, 0
#     for train_batch in tqdm(test_loader):
#         rep = rep_model(train_batch['input_ids'].to(device),
#                                                train_batch['attention_mask'].to(device))
#         level1 = level1_classification(rep)
#         level2 = level2_classification(rep)
#         level3 = level3_classification(rep)
#         level1 = level1.argmax(-1)
#         level2 = level2.argmax(-1)
#         level3 = level3.argmax(-1)

#         level1_correct += (level1 == train_batch['level1'].long().to(device)).sum().item()
#         level1_total += train_batch['level1'].shape[0]
#         level1_correct_actual += ((level1 == train_batch['level1'].long().to(device)) & (train_batch['level1'].long().to(device) != 7)).sum().item()
#         level1_total_actual += (train_batch['level1'].long().to(device) != 7).sum().item()

#         level2_correct += (level2 == train_batch['level2'].long().to(device)).sum().item()
#         level2_total += train_batch['level2'].shape[0]
#         level2_correct_actual += ((level2 == train_batch['level2'].long().to(device)) & (train_batch['level2'].long().to(device) != 14)).sum().item()
#         level2_total_actual += (train_batch['level2'].long().to(device) != 14).sum().item()


#         level3_correct += (level3 == train_batch['level3'].long().to(device)).sum().item()
#         level3_total += train_batch['level3'].shape[0]
#         level3_correct_actual += ((level3 == train_batch['level3'].long().to(device)) & (train_batch['level3'].long().to(device) != 7)).sum().item()
#         level3_total_actual += (train_batch['level3'].long().to(device) != 7).sum().item()
        
#     print(level1_correct/level1_total)
#     print(level2_correct/level2_total)
#     print(level3_correct/level3_total)
#     print(level1_correct_actual/level1_total_actual)
#     print(level2_correct_actual/level2_total_actual)
#     print(level3_correct_actual/level3_total_actual)
        

100%|██████████| 297/297 [04:36<00:00,  1.07it/s]


The loss of epoch 0 is 983.8378772735596 



100%|██████████| 297/297 [04:38<00:00,  1.07it/s]


The loss of epoch 1 is 584.7913056612015 



100%|██████████| 297/297 [04:38<00:00,  1.07it/s]

The loss of epoch 2 is 372.373140335083 



In [11]:
torch.save(rep_model.state_dict(), "rep_model2_base.pt")
torch.save(level1_classification.state_dict(), "level1_classification2_base.pt")
torch.save(level2_classification.state_dict(), "level2_classification2_base.pt")


In [10]:
level1_classification_path = "level1_classification2_base.pt"
level1_classification.load_state_dict(torch.load(level1_classification_path, map_location=device))

level2_classification_path = "level2_classification2_base.pt"
level2_classification.load_state_dict(torch.load(level2_classification_path, map_location=device))

rep_model_path = "rep_model2_base.pt"
rep_model.load_state_dict(torch.load(rep_model_path, map_location=device))
    

<All keys matched successfully>

In [13]:
from tqdm import tqdm 

rep_model.eval()
level1_classification.eval()
level2_classification.eval()
for dataload in [loader, test_loader]:
    level1_correct, level2_correct = 0, 0
    level1_correct_actual, level2_correct_actual = 0, 0
    level1_total, level2_total = 0, 0
    level1_total_actual, level2_total_actual = 0, 0
    for train_batch in tqdm(dataload):
        rep = rep_model(train_batch['input_ids'].to(device),
                                               train_batch['attention_mask'].to(device))
        level1 = level1_classification(rep)
        level2 = level2_classification(rep)
        level1 = level1.argmax(-1)
        level2 = level2.argmax(-1)

        level1_correct += (level1 == train_batch['level1'].long().to(device)).sum().item()
        level1_total += train_batch['level1'].shape[0]
        level1_correct_actual += ((level1 == train_batch['level1'].long().to(device)) & (train_batch['level1'].long().to(device) != 16)).sum().item()
        level1_total_actual += (train_batch['level1'].long().to(device) != 16).sum().item()

        level2_correct += (level2 == train_batch['level2'].long().to(device)).sum().item()
        level2_total += train_batch['level2'].shape[0]
        level2_correct_actual += ((level2 == train_batch['level2'].long().to(device)) & (train_batch['level2'].long().to(device) != 7)).sum().item()
        level2_total_actual += (train_batch['level2'].long().to(device) != 7).sum().item()

    print(level1_correct/level1_total)
    print(level2_correct/level2_total)
    print(level1_correct_actual/level1_total_actual)
    print(level2_correct_actual/level2_total_actual)

100%|██████████| 297/297 [02:10<00:00,  2.27it/s]


0.8480870217554388
0.9652100525131283
0.8480870217554388
0.9168021680216802


100%|██████████| 195/195 [01:23<00:00,  2.33it/s]

0.754096025074797
0.8704943724177233
0.754096025074797
0.7293783015034538


In [33]:
(level1 == train_batch['level1'].long().to(device)).sum()

tensor(6, device='cuda:6')

In [37]:
level2

tensor([12, 12, 11, 10,  1,  5, 11,  7], device='cuda:6')

In [38]:
train_batch['level2']

tensor([12, 12,  6, 10,  1,  5, 11, 15])

In [44]:
# comp.graphics
# comp.os.ms-windows.misc
# comp.sys.ibm.pc.hardware
# comp.sys.mac.hardware
# comp.windows.x
# rec.autos
# rec.motorcycles
# rec.sport.baseball
# rec.sport.hockey
# sci.crypt
# <sci.electronics
# sci.med
# sci.space
# misc.forsale
# talk.politics.misc
# talk.politics.guns
# talk.politics.mideast
# talk.religion.misc
# alt.atheism
# soc.religion.christian
# None
level1 = [ 
    "comp", "rec", "sci", "misc", "talk", "alt", "soc", "None"
]
level2 = [
    "os", "sys", "windows", "graphics", "motorcycles", "sport", "autos", "religion",
    "electronics", "med", "space", "forsale", "politics", "crypt", "None"
]
level2_pass = {
    "soc", "alt" 
}
level3_pass = {
    "windows", "os", "religion"
}
level3 = [
    "misc", "guns", "ibm", "mac", "baseball", "hockey", "mideast", "None"
]

hierarchy = {"comp": {"graphics", "os", "sys", "windows"}, "rec": {"auto", "motorcycles", "sport"},
            "sci": {"crypt", "electronics", "med", "space"}, "misc": {"forsale"}, 
             "talk": {"politics", "religion"}, "alt": {}, "soc": {}, "None": {},
             "windows": {}, "os": {}, "religion": {}, "politics": {"misc", "guns", "mideast"},
             "sys": {"ibm", "mac"}, "sport": {"hockey", "baseball"}
            }